#### Load modules

In [40]:
import pandas as pd
import sys, os
from gpt4all import GPT4All
from tqdm import tqdm
import re
from deep_translator import GoogleTranslator
from IPython.core.display import display, HTML

sys.path.append("\\".join(os.getcwd().split("\\")[:-1]))
%load_ext autoreload
%autoreload 2
from utils import *
from scraper import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\ma1021525\AppData\Local\Temp\ipykernel_31572\3242299578.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


#### Import data and language model

In [3]:
data = pd.read_pickle(os.path.join("/".join(os.getcwd().split("\\")[:-1]),
                                   'preprocessing','data/merged_data.pkl'))
# define the language model (Llama with 8 billion parameters is about 4.66 GB)
model = GPT4All('Meta-Llama-3-8B-Instruct.Q4_0.gguf')

#### Clean the database

In [4]:
language_dict = {'english':('EN', 'ENG'), 'french':('FR','FRA'), 'german':('DE','DEU'), 'italian':('IT','ITA'), 'not_found':('NA','NAN')}
data['lang_3'] = data.apply(lambda row: language_dict[detect_language(row['abstract'], not_found=True)][1], axis=1)
data['lang_2'] = data.apply(lambda row: language_dict[detect_language(row['abstract'], not_found=True)][0], axis=1)
# for idx, row in data.iterrows():
#     if row['abstract'] != 'nan':
#         if language_dict[detect_language(row['abstract_it'])][1] != 'ITA':
#             trnd = GoogleTranslator(source='auto', target='ita').translate(row['abstract'])
#             data.loc[idx, 'abstract_it'] = trnd
#         if language_dict[detect_language(row['abstract_de'])][1] != 'DEU':
#             trnd = GoogleTranslator(source='auto', target='de').translate(row['abstract'])
#             data.loc[idx, 'abstract_de'] = trnd
#         if language_dict[detect_language(row['abstract_fr'])][1] != 'FRA':
#             trnd = GoogleTranslator(source='auto', target='fr').translate(row['abstract'])
#             data.loc[idx, 'abstract_fr'] = trnd
#         if language_dict[detect_language(row['abstract_en'])][1] != 'ENG':
#             trnd = GoogleTranslator(source='auto', target='en').translate(row['abstract'])
#             data.loc[idx, 'abstract_en'] = trnd

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for 

#### Extract keywords with a local Llama model

In [13]:
data['abstract_w_count'] = data.apply(lambda x: len(x['abstract'].split(' ')), axis=1)
data.apply(lambda x: 'nan' if x['abstract'].startswith(('??','Es werden die Daten im Zeitraum','Link zu Metadaten:',
                                                'https:',
                                                'geo@bs.ch',
                                                'info.geoportal@be.ch',
                                                'sit@jura.ch',
                                                'geodaten@sg.ch',
                                                'info@example.com',
                                                'agi@tg.ch',
                                                'mail@lisag.ch',
                                                'gis@bd.zh.ch',
                                                'info.diffusion@vd.ch',
                                                'webgis@swisstopo.ch'))
                                                else x['abstract'], axis=1)
len(data[(data['abstract'] != 'nan') &
     (data['abstract'] != data['title']) &
     (data['abstract_w_count']>4)])

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for 

5939

In [15]:
# iterate through the dataframe
task = "Extract a list of key words comma separated without adjectives in original language from the following text: "
en_trns_task = "Translate the following list of words into a comma separated list in English: "
de_trns_task = "Übersetze die folgende Liste von Wörtern in eine durch Komma getrennte Liste auf Deutsch: "
fr_trns_task = "Traduisez la liste de mots suivante en une liste séparée par des virgules en francais: "
it_trns_task = "Traduci il seguente elenco di parole in un elenco separato da virgole in italiano: "

df_kg = pd.DataFrame()

for idx, row in tqdm(data.iterrows()):
    if idx > 100:
        continue
    trns_dict = {"ENG":en_trns_task, "DEU":de_trns_task, "ITA":it_trns_task,'FRA':fr_trns_task}
    if row['lang_3'] != 'NAN':
        del trns_dict[row['lang_3']]
    # filter rows with no abstract and use title instead
    if row['abstract'] == 'nan' or row['abstract']==row['title'] or len(row['abstract'].split(' '))<5 or row['abstract'].startswith(('??',
                                                                                                                                      'Es werden die Daten im Zeitraum',
                                                                                                                                      'Link zu Metadaten:',
                                                                                                                                      'https:',
                                                                                                                                      'geo@bs.ch',
                                                                                                                                      'info.geoportal@be.ch',
                                                                                                                                      'sit@jura.ch',
                                                                                                                                      'geodaten@sg.ch',
                                                                                                                                      'info@example.com',
                                                                                                                                      'agi@tg.ch',
                                                                                                                                      'mail@lisag.ch',
                                                                                                                                      'gis@bd.zh.ch',
                                                                                                                                      'info.diffusion@vd.ch',
                                                                                                                                      'webgis@swisstopo.ch')):
        # llama_answer = ask_llama(task, row['title'], idx)
        pass
    else:
        # generate LLM response with abstract and interpret it
        kwds = read_keyowrds(ask_llama(task, row['abstract']))

        df_kg = collect_keywords(kwds, row['lang_3'], trns_dict, df_kg)
        # for lang in [k for k in trns_dict.keys()]:
        #     llama_translation = ask_llama(trns_dict[lang], ", ".join(kwds), idx, check_response=False)
        #     translations[lang] = read_translation(llama_translation)
        # if check_length(translations, len(kwds)):
        #     df = pd.DataFrame({row['lang_3']:kwds})
        #     for lang in [k for k in trns_dict.keys()]:
        #         df[lang] = translations[lang]
        #         # data.loc[idx, 'kg_'+lang] = ','.join(translations[lang])
        #     # data.loc[idx, 'kg_'+row['lang_3']] = ','.join(kwds)
        #     df_kg = pd.concat([df_kg, df], axis=0, ignore_index=True)
        # else:
        #     print("Differing translation lengths!")

remove_rows = df_kg.applymap(lambda x: len(x.split(" "))).apply(sum, axis=1).loc[lambda x:x==4].index.tolist()
df_kg.drop(remove_rows, inplace=True)

0it [00:00, ?it/s]c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
2it [03:39, 111.17s/it]

Skipping differing translation lengths!


61it [1:12:09, 45.98s/it]

Skipping differing translation lengths!


75it [1:25:45, 56.59s/it]

...

87it [1:42:45, 84.20s/it] 

Skipping differing translation lengths!


88it [1:44:24, 88.48s/it]

Skipping differing translation lengths!


39395it [1:57:41,  5.58it/s] 


In [17]:
df_kg.to_pickle('kg_data_100.pkl')

In [30]:
kg = generate_knowledge_graph(df_kg, 'knowledge_graph', update=False)

In [47]:
if "Gefahrenkarte" in find_edges_by_language(kg, 'german'):
    print('OK')

OK


In [45]:
find_translation(kg, 'Gefahrenkarte', 'german')

["carte d'alerte",
 "carte d'aléa",
 'carta di pericolo',
 'géocarte dangereuse',
 'carta dei pericoli',
 'danger map',
 'Gefahrenkarte']